In [1]:
pip install findspark

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession

In [3]:
scala_version = "2.12"
spark_version = "3.4.1"

In [4]:
packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.7.0'
]

In [5]:
spark = SparkSession.builder.master("local").appName("kafka-example").config("spark.jars.packages", ",".join(packages)).getOrCreate()
spark

# Streaming

In [6]:
topic_name = 'Demo'
kafka_server = 'localhost:9092'

# Đọc dữ liệu từ Kafka vào DataFrame
kafkaDf = spark \
    .read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_server) \
    .option("subscribe", topic_name) \
    .option("startingOffsets", "earliest") \
    .load()

In [7]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("KafkaSparkStructuredStreaming") \
    .getOrCreate()

topic_name = 'Demo'
kafka_server = 'localhost:9092'

# Đọc dữ liệu từ Kafka vào DataFrame
kafkaDf = spark \
    .read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_server) \
    .option("subscribe", topic_name) \
    .option("startingOffsets", "earliest") \
    .load()

# Chuyển đổi dữ liệu từ chuỗi byte sang chuỗi
valueDf = kafkaDf.selectExpr("CAST(value AS STRING) as value")

# Tách các cột dựa trên dấu phẩy
splitCols = split(valueDf['value'], ',')

# Chọn các cột và đặt tên cho chúng
jsonDf = valueDf.select(splitCols[0].alias('id'),
                        splitCols[1].alias('username'),
                        splitCols[2].alias('review'))

# Chuyển đổi DataFrame từ Spark sang pandas
pandasDf = jsonDf.toPandas()

# Hiển thị dữ liệu
print(pandasDf)


      id                  username  \
0      0             Waqqas Hanafi   
1      1  Clara Hadiwijoto (Clara)   
2      2            Kristjan Malva   
3      3              colette wong   
4      4                    Hậu Lê   
..   ...                       ...   
295  295              Katie Hunter   
296  296            Marivic Arbela   
297  297          Siddhant Bahadur   
298  298        Nathalie Choubineh   
299  299            Saniya Wadikar   

                                                review  
0    "Fresh" vegetarian spring rolls were dry (the ...  
1    Food was meh. We got spring rolls but for some...  
2    Food was delightful! Restaurant was crowded so...  
3    Such a delightful restaurant with amazing Viet...  
4                               The food was delicious  
..                                                 ...  
295                     Great rooftop bar with a comfy  
296                           The food was really good  
297  The food was good but sli

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from time import sleep
from IPython.display import display, clear_output

# Phân tích cú pháp chuỗi value và chọn các cột id, username, review
batchDF = kafkaDf.selectExpr("CAST(value AS STRING) as value") \
    .withColumn("id", split(col("value"), ',')[0]) \
    .withColumn("username", split(col("value"), ',')[1]) \
    .withColumn("review", split(col("value"), ',')[2])
from time import sleep
from IPython.display import display, clear_output
for x in range(0, 2000):
    try:
        print("Showing live view refreshed every 5 seconds")
        print(f"Seconds passed: {x*5}")
        display(batchDF.toPandas())
        sleep(5)
        clear_output(wait=True)
    except KeyboardInterrupt:
        print("break")
        break
print("Live view ended...")


Showing live view refreshed every 5 seconds
Seconds passed: 20


,value,id,username,review
0,"0,Waqqas Hanafi,""Fresh"" vegetarian spring roll...",0,Waqqas Hanafi,"""Fresh"" vegetarian spring rolls were dry (the ..."
1,"1,Clara Hadiwijoto (Clara),Food was meh. We go...",1,Clara Hadiwijoto (Clara),Food was meh. We got spring rolls but for some...
2,"2,Kristjan Malva,Food was delightful! Restaura...",2,Kristjan Malva,Food was delightful! Restaurant was crowded so...
3,"3,colette wong,Such a delightful restaurant wi...",3,colette wong,Such a delightful restaurant with amazing Viet...
4,"4,Hậu Lê,The food was delicious, i know everyo...",4,Hậu Lê,The food was delicious
...,...,...,...,...
295,"295,Katie Hunter,Great rooftop bar with a comf...",295,Katie Hunter,Great rooftop bar with a comfy
296,"296,Marivic Arbela,The food was really good, a...",296,Marivic Arbela,The food was really good
297,"297,Siddhant Bahadur,The food was good but sli...",297,Siddhant Bahadur,The food was good but slightly pricey and the ...
298,"298,Nathalie Choubineh,Good food. Friendly peo...",298,Nathalie Choubineh,Good food. Friendly people. But pricy and the ...


break
Live view ended...


# Tiền xử lý 

In [10]:
from pyspark.sql.functions import split, explode

# Chia cột "review" thành một mảng các câu
jsonDf = jsonDf.withColumn("sentences", split(jsonDf["review"], r'(?<=[.!?])\s+'))

# Tách các câu thành các hàng riêng biệt và giữ tên cột là "review"
jsonDf_exploded = jsonDf.withColumn("review", explode(jsonDf["sentences"]))

# Chọn các cột cần thiết và bỏ cột "sentences"
jsonDf = jsonDf_exploded.select("id", "username", "review")

# Hiển thị kết quả
jsonDf.show(truncate=False)

+---+------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |username                |review                                                                                                                                                        |
+---+------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0  |Waqqas Hanafi           |"Fresh" vegetarian spring rolls were dry (the skin was like chewing through leathery dry paper) and                                                           |
|1  |Clara Hadiwijoto (Clara)|Food was meh.                                                                                                                                                 |
|1  |Clara Hadiwijoto (Clara)|We got spring rolls 

# Model 1

In [11]:
!pip install transformers torch pyspark tensorflow


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
!pip install --upgrade tensorflow
!pip install --upgrade keras


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
import logging
# Khởi tạo Spark Session
spark = SparkSession.builder \
    .appName("Stage1_Inference") \
    .getOrCreate()

# Định nghĩa các hàm inference của bạn
def get_prediction_stage1(sentence):
    import numpy as np
    import tensorflow as tf
    #from tensorflow.keras.preprocessing.text import Tokenizer
    from tensorflow.keras.preprocessing.sequence import pad_sequences
    import joblib
    import pickle


    logging.info(f"Processing sentence: {sentence}")

    # Load the model
    model = tf.keras.models.load_model("./LSTM_rest.h5")

    # Load the tokenizer
    #tokenizer = joblib.load('/content/drive/MyDrive/Study/Năm 3/DS200 - Big data/project/model/tokenizer.pkl')
    with open('./tokenizer.pkl', 'rb') as f:
      tokenizer = pickle.load(f)

    # Tokenize and pad the input sentence
    max_sequence_length = 100
    sentence_tokenized = tokenizer.texts_to_sequences([sentence])
    logging.info(f"Tokenized sentence: {sentence_tokenized}")
    sentence_padded = pad_sequences(sentence_tokenized, maxlen=max_sequence_length, padding='post')
    logging.info(f"Padded sentence: {sentence_padded}")

    # Predict the label
    prediction = model.predict(sentence_padded)
    logging.info(f"Prediction: {prediction}")

    # Convert the predicted probabilities to class labels
    predicted_label = 1 if prediction[0][1] > 0.5 else 0

    return int(predicted_label)

# Đăng ký hàm inference như một UDF
stage1_udf = udf(get_prediction_stage1, IntegerType())

In [14]:
import pandas as pd

In [15]:
# Tách các cột dựa trên dấu phẩy
splitCols = split(valueDf['value'], ',')

# Chọn các cột và đặt tên cho chúng
#jsonDf = valueDf.select(splitCols[0].alias('id'),
#                        splitCols[1].alias('username'),
#                        splitCols[2].alias('review'))

# Đăng ký hàm inference như một UDF
stage1_udf = udf(get_prediction_stage1, IntegerType())

# Áp dụng UDF lên DataFrame để dự đoán
df_with_predictions = jsonDf.withColumn("prediction_stage1", stage1_udf(jsonDf["review"]))


In [17]:
df_with_predictions

DataFrame[id: string, username: string, review: string, prediction_stage1: int]

In [18]:
review_df = df_with_predictions.select("review")

In [19]:
review_df.show(truncate = False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|review                                                                                                                                                        |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|"Fresh" vegetarian spring rolls were dry (the skin was like chewing through leathery dry paper) and                                                           |
|Food was meh.                                                                                                                                                 |
|We got spring rolls but for some reason it feels so dry Place was nicely decorated Price was written in VND and USD and imo it was definitely overpriced      |
|Food was delightful!             

In [20]:
jsonDf.count()

701

In [21]:
import pandas as pd
# Chọn các cột cần thiết từ jsonDf
data = jsonDf.select("username", "review").collect()

# Chuyển đổi dữ liệu thành định dạng Pandas DataFrame
columns = ["username", "review"]
df_pandas = pd.DataFrame(data, columns=columns)

# Áp dụng inference cho cột "review" và thêm cột "prediction" vào Pandas DataFrame
predictions = [get_prediction_stage1(row['review']) for row in data]
df_pandas['prediction_stage1'] = predictions

# Tạo Spark DataFrame từ Pandas DataFrame
df_spark = spark.createDataFrame(df_pandas)

# Hiển thị kết quả
df_spark.show(truncate=False)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 517ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 315ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 358ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 336ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 294ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 376ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step
+------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+
|username                |review                                                                                                                                                        |prediction_stage1|
+------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+
|Waqqas Hanafi           |"Fresh" vegetarian spring rolls were dry (the skin was like chewing through leathery dry paper) and                                                           |1                |
|Clara Hadiwijoto (Clara)|Food was meh.                                                                                                          

In [22]:
df_spark.show(truncate=True)

+--------------------+--------------------+-----------------+
|            username|              review|prediction_stage1|
+--------------------+--------------------+-----------------+
|       Waqqas Hanafi|"Fresh" vegetaria...|                1|
|Clara Hadiwijoto ...|       Food was meh.|                1|
|Clara Hadiwijoto ...|We got spring rol...|                1|
|      Kristjan Malva|Food was delightful!|                1|
|      Kristjan Malva|Restaurant was cr...|                1|
|      Kristjan Malva|Our waiter did no...|                1|
|      Kristjan Malva|I think it was so...|                1|
|      Kristjan Malva|Otherwise would r...|                1|
|      Kristjan Malva|Prices are very e...|                1|
|        colette wong|Such a delightful...|                1|
|        colette wong|We ordered Vietna...|                1|
|              Hậu Lê|The food was deli...|                1|
|                 T C|Dined here 3 time...|                1|
|       

# Model 2

In [23]:
df_filtered = df_with_predictions.filter(col("prediction_stage1") == 1)

In [24]:
df_spark.count()

701

In [25]:
df_spark = df_spark.filter(col("prediction_stage1") == 1)

In [26]:
df_spark.count()

688

In [110]:
#df_filtered = df_filtered.select("review")

In [27]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Khởi tạo Spark Session
spark = SparkSession.builder \
    .appName("Stage2_Inference") \
    .getOrCreate()

# Định nghĩa các hàm inference của bạn
def get_prediction_stage2(review):
    model_id = 'ThuyNT03/DS200-big_train-res16-T5'
    tokenizer_T5 = AutoTokenizer.from_pretrained(model_id)
    model_T5 = AutoModelForSeq2SeqLM.from_pretrained(model_id)

    max_input_length = 256
    max_output_length = 256

    input_ids = tokenizer_T5(review, max_length=max_input_length, return_tensors="pt", padding="max_length", truncation=True).input_ids
    outputs = model_T5.generate(input_ids=input_ids, max_new_tokens=max_output_length, do_sample=True)
    pred = tokenizer_T5.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]

    return pred

# Đăng ký hàm inference như một UDF
stage2_udf = udf(get_prediction_stage2, StringType())

# Áp dụng UDF lên df_filtered từ Model 1
df_with_predictions_stage2 = df_filtered.withColumn("prediction_stage2", stage2_udf(df_filtered["review"]))

In [28]:
display(df_with_predictions_stage2)

DataFrame[id: string, username: string, review: string, prediction_stage1: int, prediction_stage2: string]

In [29]:
import pandas as pd

# Chuyển đổi dữ liệu thành định dạng Pandas DataFrame
columns = [ "username", "review"]
data_list = [{'username': row['username'], 'review': row['review']} for row in df_spark.collect()]
df_pandas = pd.DataFrame(data_list, columns=columns)

# Áp dụng inference cho cột "review" và thêm cột "prediction" vào Pandas DataFrame
predictions = [get_prediction_stage2(row['review']) for row in data_list]
df_pandas['prediction_stage2'] = predictions

# Tạo Spark DataFrame từ Pandas DataFrame
df_spark_with_predictions = spark.createDataFrame(df_pandas)

# Hiển thị kết quả
df_spark_with_predictions.show(truncate=False)


+------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------+
|username                |review                                                                                                                                                  |prediction_stage2                                                                                                                                           |
+------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------

In [30]:
df_spark_with_predictions.show(truncate=True)

+--------------------+--------------------+--------------------+
|            username|              review|   prediction_stage2|
+--------------------+--------------------+--------------------+
|       Waqqas Hanafi|"Fresh" vegetaria...|vegetarian spring...|
|Clara Hadiwijoto ...|       Food was meh.|Food, food qualit...|
|Clara Hadiwijoto ...|We got spring rol...|Spring rolls, foo...|
|      Kristjan Malva|Food was delightful!|Food, food qualit...|
|      Kristjan Malva|Restaurant was cr...|Restaurant, ambie...|
|      Kristjan Malva|Our waiter did no...|waiter, service g...|
|      Kristjan Malva|I think it was so...|NULL, service gen...|
|      Kristjan Malva|Otherwise would r...|Ben Thanh Market,...|
|      Kristjan Malva|Prices are very e...|NULL, restaurant ...|
|        colette wong|Such a delightful...|restaurant, ambie...|
|        colette wong|We ordered Vietna...|Vietnamese rolls,...|
|              Hậu Lê|The food was deli...|food, food qualit...|
|                 T C|Din

In [31]:
# Hiển thị chỉ cột "prediction_stage2"
df_spark_with_predictions.select("prediction_stage2").show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------+
|prediction_stage2                                                                                                                                           |
+------------------------------------------------------------------------------------------------------------------------------------------------------------+
|vegetarian spring rolls, food quality, negative, dry                                                                                                        |
|Food, food quality, negative, meh                                                                                                                           |
|Spring rolls, food quality, negative, dry; Place, ambience general, positive, nicely decorated; Place, food prices, negative, overpriced                    |
|Food, food quality, positive, delightful     

In [34]:
from pyspark.sql import functions as F

# Gộp các mẫu có cùng "username" và tính tổng prediction_stage2
df_aggregated = df_spark_with_predictions.groupBy("username") \
    .agg(F.concat_ws(" | ", F.collect_list("prediction_stage2")).alias("prediction_stage2"))

# Chỉ lấy lại cột "username" và "prediction_stage2"
df_final = df_aggregated.select("username", "prediction_stage2")

# Hiển thị kết quả
df_final.show(truncate=False)


+----------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|username        |prediction_stage2                                                                                                                                                                                                                                                                                                                                                             |
+----------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [35]:
df_final.count()

299